In [1]:
import pandas as pd
pd.set_option('max_colwidth', 800)
pd.set_option('display.max_rows', 500)

/Users/sathvik/nfl-pbp/lib/python3.9/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
data = pd.DataFrame()
i_data = pd.read_csv('./play_by_play_2020.csv.gz', compression='gzip', low_memory=False)
data = data.append(i_data, sort=True)
data.reset_index(drop=True, inplace=True)


In [3]:
import numpy as np
from sklearn.linear_model import LinearRegression


In [4]:
data['down'] = np.nan_to_num(data['down'])
#data['yards_gained'] = np.nan_to_num(data['yards_gained'])
# data['rusher'] = data['rusher'].fillna('')
#data['incomplete_pass'] = np.nan_to_num(data['incomplete_pass'], nan=-1)
#data['punter_player_name'] = data['punter_player_name'].fillna('')
#data['kick_distance'] = np.nan_to_num(data['kick_distance'])
# data['ydstogo'] = np.nan_to_num(data['ydstogo'])



In [5]:
# from https://stackoverflow.com/a/62085741
def undummify(df, prefix_sep="_"):
    cols2collapse = {
        item.split(prefix_sep)[0]: (prefix_sep in item) for item in df.columns
    }
    series_list = []
    for col, needs_to_collapse in cols2collapse.items():
        if needs_to_collapse:
            undummified = (
                df.filter(like=col)
                .idxmax(axis=1)
                .apply(lambda x: x.split(prefix_sep, maxsplit=1)[1])
                .rename(col)
            )
            series_list.append(undummified)
        else:
            series_list.append(df[col])
    undummified_df = pd.concat(series_list, axis=1)
    return undummified_df

In [6]:
featured_cols_play = ['posteam', 'down']
x_play = pd.get_dummies(data.loc[:, featured_cols_play])
x_play.shape
print(x_play.columns)

Index(['down', 'posteam_ARI', 'posteam_ATL', 'posteam_BAL', 'posteam_BUF',
       'posteam_CAR', 'posteam_CHI', 'posteam_CIN', 'posteam_CLE',
       'posteam_DAL', 'posteam_DEN', 'posteam_DET', 'posteam_GB',
       'posteam_HOU', 'posteam_IND', 'posteam_JAX', 'posteam_KC', 'posteam_LA',
       'posteam_LAC', 'posteam_LV', 'posteam_MIA', 'posteam_MIN', 'posteam_NE',
       'posteam_NO', 'posteam_NYG', 'posteam_NYJ', 'posteam_PHI',
       'posteam_PIT', 'posteam_SEA', 'posteam_SF', 'posteam_TB', 'posteam_TEN',
       'posteam_WAS'],
      dtype='object')


In [7]:
output_cols_play = ['play_type', 'passer', 'rusher', 'receiver', 'kicker_player_name', 'punter_player_name']
y_play = pd.get_dummies(data.loc[:, output_cols_play])
y_play.shape

(43176, 1071)

In [8]:
model_play = LinearRegression()
model_play.fit(x_play,y_play)

LinearRegression()

In [9]:
row = [1, 0, 0, 1, 0,
       0, 0, 0, 0,
       0, 0, 0, 0,
       0, 0, 0, 0, 0,
       0, 0, 0, 0, 0,
       0, 0, 0, 0,
       0, 0, 0, 0, 0,
       0]
result = model_play.predict([row])
df = pd.DataFrame(result, columns=pd.Series(list(y_play.columns)))
undummify(df)

,play,passer,rusher,receiver,kicker,punter
0,type_run,L.Jackson,G.Edwards,M.Brown,player_name_J.Tucker,player_name_S.Koch


In [10]:
featured_cols_play_alt = ['posteam', 'down', 'ydstogo']
x_play_alt = pd.get_dummies(data.loc[:, featured_cols_play_alt])
output_cols_play_alt = ['play_type', 'passer', 'rusher', 'receiver', 'kicker_player_name', 'punter_player_name']
y_play_alt = pd.get_dummies(data.loc[:, output_cols_play_alt])
model_play_alt = LinearRegression()
model_play_alt.fit(x_play_alt,y_play_alt)

LinearRegression()

In [11]:
row_alt = [3, 4, 0, 0, 0, 0,
       0, 0, 0, 0,
       0, 0, 0, 0,
       0, 0, 0, 0, 0,
       0, 0, 0, 0, 0,
       0, 0, 0, 0,
       0, 0, 1, 0, 0,
       0]
result_alt = model_play_alt.predict([row_alt])
df_alt = pd.DataFrame(result_alt, columns=pd.Series(list(y_play_alt.columns)))
undummify(df_alt)

,play,passer,rusher,receiver,kicker,punter
0,type_pass,N.Mullens,R.Mostert,B.Aiyuk,player_name_R.Gould,player_name_M.Wishnowsky


In [12]:
print(result.size)

1071


In [13]:
# df = pd.DataFrame({ x: pd.Series(result[0]), y: pd.Series(list(y.columns)) })
df = pd.DataFrame(result, columns=pd.Series(list(y_play.columns)))
undummify(df)

,play,passer,rusher,receiver,kicker,punter
0,type_run,L.Jackson,G.Edwards,M.Brown,player_name_J.Tucker,player_name_S.Koch


In [14]:
data['rusher'] = data['rusher'].fillna('')
data['yards_gained'] = np.nan_to_num(data['yards_gained'])

In [15]:
featured_cols_rush = ['rusher', 'down']
x_rush = pd.get_dummies(data.loc[:, featured_cols_rush])
x_rush.shape


(43176, 345)

In [16]:
output_cols_rush = ['yards_gained']
y_rush = pd.get_dummies(data.loc[:, output_cols_rush])


In [17]:
model_rush = LinearRegression()
model_rush.fit(x_rush, y_rush)

LinearRegression()

In [18]:
print(x_rush.columns)

Index(['down', 'rusher_', 'rusher_A.Abdullah', 'rusher_A.Armah',
       'rusher_A.Brown', 'rusher_A.Cole', 'rusher_A.Collins',
       'rusher_A.Cooper', 'rusher_A.Dalton', 'rusher_A.Dillon',
       ...
       'rusher_T.Tagovailoa', 'rusher_T.Taylor', 'rusher_T.Wesco',
       'rusher_T.Williams', 'rusher_T.Yeldon', 'rusher_V.Jefferson',
       'rusher_W.Fuller', 'rusher_W.Gallman', 'rusher_Z.Ertz',
       'rusher_Z.Moss'],
      dtype='object', length=345)


In [19]:
row_rusher = [1]
for col in list(x_rush.columns)[1:]:
    if col == 'rusher_A.Jones':
        row_rusher.append(1)
    else:
        row_rusher.append(0)

model_rush.predict([row_rusher])

array([[5.23262787]])

In [20]:
data['incomplete_pass'] = np.nan_to_num(data['incomplete_pass'], nan=-1)

In [21]:
featured_cols_pass = ['down','passer', 'receiver',]
x_pass = pd.get_dummies(data.loc[:, featured_cols_pass])
x_pass.shape

(43176, 611)

In [22]:
output_cols_pass = ['yards_gained', 'incomplete_pass']
y_pass = pd.get_dummies(data.loc[:, output_cols_pass])

In [23]:
model_pass = LinearRegression()
model_pass.fit(x_pass, y_pass)

LinearRegression()

In [24]:
print(x_pass.columns)

Index(['down', 'passer_A.Dalton', 'passer_A.Erickson', 'passer_A.Humphries',
       'passer_A.Lee', 'passer_A.McCarron', 'passer_A.Rodgers',
       'passer_A.Smith', 'passer_B.Allen', 'passer_B.Colquitt',
       ...
       'receiver_V.McDonald', 'receiver_V.Smith', 'receiver_W.Dissly',
       'receiver_W.Fuller', 'receiver_W.Gallman', 'receiver_W.Snead',
       'receiver_Z.Ertz', 'receiver_Z.Jones', 'receiver_Z.Moss',
       'receiver_Z.Pascal'],
      dtype='object', length=611)


In [25]:
row_pass = [1]
for col in list(x_pass.columns)[1:]:
    if col == 'passer_P.Mahomes':
        row_pass.append(1)
    elif col == 'receiver_T.Kelce':
        row_pass.append(1)
    else:
        row_pass.append(0)

model_pass.predict([row_pass])

array([[9.24747843, 0.19847472]])

In [26]:
data['punter_player_name'] = data['punter_player_name'].fillna('')
data['kick_distance'] = np.nan_to_num(data['kick_distance'])

In [27]:
featured_cols_punt = ['punter_player_name',]
x_punt = pd.get_dummies(data.loc[:, featured_cols_punt])
x_punt.shape

(43176, 40)

In [28]:
output_cols_punt = ['kick_distance']
y_punt = pd.get_dummies(data.loc[:, output_cols_punt])

In [29]:
model_punt = LinearRegression()
model_punt.fit(x_punt, y_punt)

LinearRegression()

In [30]:
print(x_punt.columns)

Index(['punter_player_name_', 'punter_player_name_A.Cole',
       'punter_player_name_A.Lee', 'punter_player_name_A.Rosas',
       'punter_player_name_B.Anger', 'punter_player_name_B.Colquitt',
       'punter_player_name_B.Kern', 'punter_player_name_B.Mann',
       'punter_player_name_B.Pinion', 'punter_player_name_C.Bojorquez',
       'punter_player_name_C.Johnston', 'punter_player_name_C.Jones',
       'punter_player_name_D.Colquitt', 'punter_player_name_H.Niswander',
       'punter_player_name_J.Bailey', 'punter_player_name_J.Berry',
       'punter_player_name_J.Charlton', 'punter_player_name_J.Elliott',
       'punter_player_name_J.Fox', 'punter_player_name_J.Gillan',
       'punter_player_name_J.Hekker', 'punter_player_name_J.Scott',
       'punter_player_name_K.Fairbairn', 'punter_player_name_K.Huber',
       'punter_player_name_L.Cooke', 'punter_player_name_M.Dickson',
       'punter_player_name_M.Haack', 'punter_player_name_M.Wishnowsky',
       'punter_player_name_P.O'Donnell'

In [31]:
row_punt = []
for col in list(x_punt.columns):
    if col == 'punter_player_name_T.Way':
        row_punt.append(1)
    else:
        row_punt.append(0)

model_punt.predict([row_punt])

array([[45.60546875]])

In [32]:
featured_cols_kick = ['kicker_player_name', 'kick_distance']
x_kick = pd.get_dummies(data.loc[:, featured_cols_kick])
x_kick.shape

(43176, 71)

In [33]:
output_cols_kick = ['extra_point_result', 'field_goal_result']
y_kick = pd.get_dummies(data.loc[:, output_cols_kick])

In [34]:
model_kick = LinearRegression()
model_kick.fit(x_kick, y_kick)

LinearRegression()

In [35]:
print(x_kick.columns)

Index(['kick_distance', 'kicker_player_name_A.Brewer',
       'kicker_player_name_A.Lee', 'kicker_player_name_A.Rosas',
       'kicker_player_name_A.Seibert', 'kicker_player_name_B.Anger',
       'kicker_player_name_B.Colquitt', 'kicker_player_name_B.Kern',
       'kicker_player_name_B.Mann', 'kicker_player_name_B.McManus',
       'kicker_player_name_B.Pinion', 'kicker_player_name_B.Wright',
       'kicker_player_name_C.Boswell', 'kicker_player_name_C.Johnston',
       'kicker_player_name_C.Jones', 'kicker_player_name_C.McLaughlin',
       'kicker_player_name_C.Parkey', 'kicker_player_name_C.Santos',
       'kicker_player_name_D.Bailey', 'kicker_player_name_D.Carlson',
       'kicker_player_name_D.Hopkins', 'kicker_player_name_E.Fry',
       'kicker_player_name_G.Gano', 'kicker_player_name_G.Zuerlein',
       'kicker_player_name_H.Butker', 'kicker_player_name_J.Bailey',
       'kicker_player_name_J.Brown', 'kicker_player_name_J.Elliott',
       'kicker_player_name_J.Fox', 'kicker_playe

In [36]:
row_kick = [33]
for col in list(x_kick.columns)[1:]:
    if col == 'kicker_player_name_R.Blankenship':
        row_kick.append(1)
    else:
        row_kick.append(0)

prediction = list(model_kick.predict([row_kick]))

for i in range(len(list(y_kick.columns))):
    print(list(y_kick.columns)[i], prediction[0][i])

extra_point_result_blocked 6.349090131926459e-05
extra_point_result_failed 0.02158988628767188
extra_point_result_good 0.4412548698220293
field_goal_result_blocked 0.00010100413671603473
field_goal_result_made 0.3243749456229952
field_goal_result_missed 0.033451578344567336


In [37]:
print(y_kick.columns)

Index(['extra_point_result_blocked', 'extra_point_result_failed',
       'extra_point_result_good', 'field_goal_result_blocked',
       'field_goal_result_made', 'field_goal_result_missed'],
      dtype='object')


In [38]:
data[data['field_goal_result'] == 'made'].head(1)['kick_distance']

7    52.0
Name: kick_distance, dtype: float64

In [39]:
def input_row_for_play_type_model(down, ydstogo, team_name):
    team_keys = ['posteam_ARI', 'posteam_ATL', 'posteam_BAL', 'posteam_BUF',
       'posteam_CAR', 'posteam_CHI', 'posteam_CIN', 'posteam_CLE',
       'posteam_DAL', 'posteam_DEN', 'posteam_DET', 'posteam_GB',
       'posteam_HOU', 'posteam_IND', 'posteam_JAX', 'posteam_KC', 'posteam_LA',
       'posteam_LAC', 'posteam_LV', 'posteam_MIA', 'posteam_MIN', 'posteam_NE',
       'posteam_NO', 'posteam_NYG', 'posteam_NYJ', 'posteam_PHI',
       'posteam_PIT', 'posteam_SEA', 'posteam_SF', 'posteam_TB', 'posteam_TEN',
       'posteam_WAS']
    output_row = [down, ydstogo]
    output_row.extend([1 if item.endswith(team_name) else 0 for item in team_keys])
    return output_row
input_row_for_play_type_model(1, 10, 'ARI')

[1,
 10,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [40]:
import random

def series_sim(team_name, starting_pos):
    series_yard_line = starting_pos
    series_time_taken = 0
    series_ydstogo = 10
    for down in range(1,5):
        play_model_input = input_row_for_play_type_model(down, series_ydstogo, team_name)
        play_prediction = model_play_alt.predict([play_model_input])
        df = pd.DataFrame(play_prediction, columns=pd.Series(list(y_play_alt.columns)))

        play_type_result = undummify(df)
        play_call = play_type_result.loc[0].play
        play_passer = play_type_result.loc[0].passer
        play_receiver = play_type_result.loc[0].receiver
        play_rusher = play_type_result.loc[0].rusher
        
        random_running_back_selection = random.random()
        if (random_running_back_selection > 0.8):
            df.loc[0, 'rusher_'+ play_rusher] = -1
            play_rusher = undummify(df).loc[0].rusher
        
        # TODO make this per team, e.g. (# of plays where receiver is Kelce and passer is Mahomes/total # of plays where passer is Mahomes)
        # randomly select WR2
        random_wide_receiver_selection = random.random()
        if (random_wide_receiver_selection > 0.5):
            df.loc[0, 'receiver_' + play_receiver] = -1
            play_receiver = undummify(df).loc[0].receiver
            
        # randomly select WR3
        if (random_wide_receiver_selection > 0.75):
            df.loc[0, 'receiver_' + play_receiver] = -1
            play_receiver = undummify(df).loc[0].receiver
        
        
        # randomly select RB as pass catcher
        if (random_wide_receiver_selection > 0.9):
            play_receiver = play_rusher
        
        multiplier_list = [random.randrange(0, 2)] * 40 + [random.randrange(2, 3)] * 10 + [random.randrange(3, 4)] * 3 + [random.randrange(4, 5)] * 1
        
        print(play_call, play_passer, play_receiver, play_rusher)
        if play_call == 'type_pass':
            input_pass = [down]
            for col in list(x_pass.columns)[1:]:
                if col == 'passer_' + (play_passer):
                    input_pass.append(1)
                elif col == 'receiver_' +(play_receiver):
                    input_pass.append(1)
                else:
                    input_pass.append(0)
            receiving_yards_gained, catch_drop_probability = model_pass.predict([input_pass])[0]
            rand_value = random.random()
            catch_drop_probability_multiplier = 0 if rand_value <= catch_drop_probability else 1
            total_receiving_yards_gained = round((receiving_yards_gained * catch_drop_probability_multiplier * random.choice(multiplier_list)))
            series_yard_line += total_receiving_yards_gained
            series_ydstogo = 10 - total_receiving_yards_gained
            if series_yard_line >= starting_pos + 10:
                break
        elif play_call == 'type_run':
            input_rush = [down]
            for col in list(x_rush.columns)[1:]:
                if col == 'rusher_'+ play_rusher:
                    input_rush.append(1)
                else:
                    input_rush.append(0)
            rush_yards_gained = random.choice(multiplier_list) * model_rush.predict([input_rush])[0][0]
            series_yard_line += round(rush_yards_gained)
            series_ydstogo = 10 - round(rush_yards_gained)
            if series_yard_line >= starting_pos + 10:
                break
        series_time_taken = random.randint(30, 90)
    return {
        'series_end_line': series_yard_line,
        'series_result': 'first_down' if series_yard_line >= starting_pos + 10 else 'turnover',
        'series_time_taken_sec': series_time_taken
    }        
    
series_sim('ARI', 25)

type_pass K.Murray D.Hopkins K.Drake
type_run K.Murray C.Kirk K.Drake


{'series_end_line': 41,
 'series_result': 'first_down',
 'series_time_taken_sec': 87}

In [41]:
def drive_sim(team_name, starting_pos):
    #continue_drive = true
    drive_yard_line = starting_pos
    drive_total_time_sec = 0
    while True:
        result_current_series = series_sim(team_name, drive_yard_line)
        drive_yard_line = result_current_series['series_end_line']
        drive_total_time_sec += result_current_series['series_time_taken_sec']
        if drive_yard_line >= 100 or result_current_series['series_result'] == 'turnover':
                break
        
    return {
        'drive_result': 'touchdown' if drive_yard_line >= 100 else 'turnover',
        'points_gained': 7 if drive_yard_line >= 100 else 0,
        'drive_total_time_sec': drive_total_time_sec 
    }
drive_sim('CLE', 25)    

type_pass B.Mayfield J.Landry K.Hunt
type_run B.Mayfield R.Higgins K.Hunt
type_pass B.Mayfield A.Hooper K.Hunt
type_pass B.Mayfield A.Hooper K.Hunt
type_pass B.Mayfield J.Landry K.Hunt
type_pass B.Mayfield R.Higgins K.Hunt
type_pass B.Mayfield A.Hooper K.Hunt
type_pass B.Mayfield A.Hooper K.Hunt
type_pass B.Mayfield J.Landry K.Hunt
type_pass B.Mayfield R.Higgins K.Hunt


{'drive_result': 'touchdown', 'points_gained': 7, 'drive_total_time_sec': 177}

In [43]:
def game_sim(home_team, away_team):
    starting_field_position = 25
    home_team_points = 0
    away_team_points = 0
    total_game_time_sec = 0
    pos_team = home_team
    while total_game_time_sec <= 60*60:
        drive_simulation = drive_sim(pos_team, starting_field_position)
        print(pos_team, drive_simulation)
        if pos_team == home_team:
            home_team_points += drive_simulation['points_gained']
            pos_team = away_team
        else:
            away_team_points += drive_simulation['points_gained']
            pos_team = home_team
        total_game_time_sec += drive_simulation['drive_total_time_sec']
    return {
        'home_team_points': home_team_points,
        'away_team_points': away_team_points,
        'winning_team': home_team if home_team_points > away_team_points else away_team
    }
game_sim('DEN', 'CIN')

type_pass D.Lock M.Gordon M.Gordon
type_pass D.Lock J.Jeudy P.Lindsay
type_pass D.Lock T.Patrick M.Gordon
type_pass D.Lock J.Jeudy M.Gordon
DEN {'drive_result': 'turnover', 'points_gained': 0, 'drive_total_time_sec': 33}
type_pass J.Burrow T.Higgins G.Bernard
type_pass J.Burrow A.Green J.Mixon
type_pass J.Burrow T.Boyd G.Bernard
type_pass J.Burrow G.Bernard G.Bernard
type_pass J.Burrow A.Green G.Bernard
CIN {'drive_result': 'turnover', 'points_gained': 0, 'drive_total_time_sec': 37}
type_pass D.Lock N.Fant M.Gordon
type_pass D.Lock N.Fant P.Lindsay
type_pass D.Lock J.Jeudy M.Gordon
type_pass D.Lock T.Patrick M.Gordon
type_pass D.Lock N.Fant M.Gordon
type_pass D.Lock N.Fant M.Gordon
DEN {'drive_result': 'turnover', 'points_gained': 0, 'drive_total_time_sec': 140}
type_pass J.Burrow T.Higgins G.Bernard
type_pass J.Burrow T.Higgins G.Bernard
type_pass J.Burrow T.Boyd J.Mixon
type_pass J.Burrow T.Boyd G.Bernard
type_pass J.Burrow T.Higgins G.Bernard
type_pass J.Burrow G.Bernard G.Bernard
C

type_pass J.Burrow A.Green G.Bernard
type_pass J.Burrow T.Higgins G.Bernard
type_pass J.Burrow T.Boyd G.Bernard
type_pass J.Burrow T.Higgins G.Bernard
CIN {'drive_result': 'turnover', 'points_gained': 0, 'drive_total_time_sec': 90}
type_pass D.Lock M.Gordon M.Gordon
type_pass D.Lock N.Fant P.Lindsay
type_pass D.Lock J.Jeudy M.Gordon
type_pass D.Lock J.Jeudy M.Gordon
DEN {'drive_result': 'turnover', 'points_gained': 0, 'drive_total_time_sec': 73}
type_pass J.Burrow T.Higgins G.Bernard
type_pass J.Burrow T.Higgins G.Bernard
type_pass J.Burrow J.Mixon J.Mixon
type_pass J.Burrow T.Higgins J.Mixon
type_pass J.Burrow T.Higgins G.Bernard
CIN {'drive_result': 'turnover', 'points_gained': 0, 'drive_total_time_sec': 48}
type_pass D.Lock T.Patrick M.Gordon
type_pass D.Lock J.Jeudy M.Gordon
type_pass D.Lock J.Jeudy M.Gordon
type_pass D.Lock N.Fant M.Gordon
type_pass D.Lock J.Jeudy M.Gordon
type_pass D.Lock J.Jeudy P.Lindsay
type_pass D.Lock J.Jeudy M.Gordon
DEN {'drive_result': 'turnover', 'points

{'home_team_points': 7, 'away_team_points': 7, 'winning_team': 'CIN'}

In [47]:
from joblib import dump, load
dump(model_play_alt, 'model_play_alt.joblib')
dump(model_pass, 'model_pass.joblib')
dump(model_rush, 'model_rush.joblib')
dump(model_punt, 'model_punt.joblib')
dump(model_kick, 'model_kick.joblib')
dump(y_play_alt, 'y_play_alt.joblib')
dump(x_pass, 'x_pass.joblib')
dump(x_rush, 'x_rush.joblib')

['x_rush.joblib']